In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yfin
import math
import matplotlib_inline.backend_inline 
import statsmodels.api as sm
from pandas_datareader import data as pdr
import plotly.graph_objects as go

matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'png')

plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"

In [ ]:
# 2.1 Returns

stock_ids = ('GOOG', 'AAPL', 'TSLA')
yfin.pdr_override()
stock_data = pdr.get_data_yahoo(stock_ids, datetime(2013, 2, 1),
                                datetime(2023, 2, 1))["Adj Close"]
net_returns = 100 * (stock_data - stock_data.shift(1)) / stock_data.shift(1)
net_returns = net_returns.dropna()

net_returns.head()

mu = net_returns.mean(axis=0) * 252.0 
mu

In [ ]:
# 2.1 Covariance Matrix 

cov = net_returns.cov() * 252.0 
cov

In [ ]:
# 2.1 Efficient Frontier 

n_pf = 10000 #number of portfolios in our simulation
sample_pf = 0.5 * np.random.normal(size=(len(stock_ids), n_pf))
mu_pf = np.array(mu) @ sample_pf
var_pf = (sample_pf.T @ np.array(cov) * sample_pf.T).sum(-1)
ef_flag = np.zeros_like(mu_pf)
for i in range(n_pf):
  if all(mu_pf[var_pf < var_pf[i]] < mu_pf[i]):
    ef_flag[i] = 1

fig = go.Figure()
fig.add_trace(
    go.Scatter(mode='markers',
               x=np.sqrt(var_pf),
               y=mu_pf,
               marker=dict(color='black', size=1),
               showlegend=False))
fig.add_trace(
    go.Scatter(mode='markers',
               x=np.sqrt(var_pf[np.nonzero(ef_flag)]),
               y=mu_pf[np.nonzero(ef_flag)],
               marker=dict(color='blue', size=2),
               name='Optimal'))
fig.add_trace(
    go.Scatter(mode='markers',
               x=np.sqrt(np.diag(np.array(cov))),
               y=np.array(mu),
               marker=dict(color='crimson', size=8),
               name='Individual'))
fig.update_layout(xaxis_title="Volatility",
                  yaxis_title="Mean",
                  height=600,
                  width=700)
fig.update_xaxes(range=[-3, 50])
fig.update_yaxes(range=[-50, 50])
fig.show()


In [ ]:
# 3.1 High Dimensional Covariance Matrix Estimation: p = 3
mean = np.zeros(3)
cov = np.identity(3)

simulations = np.random.multivariate_normal(mean, cov, size=1000)
cov_matrix = np.cov(simulations.T)

np.linalg.eigvals(cov_matrix)

In [ ]:
# 3.2 High Dimensional Covariance Matrix Estimation: p = 1000
mean_1000 = np.zeros(1000)
cov_1000 = np.identity(1000)

simulations_1000 = np.random.multivariate_normal(mean_1000, cov_1000, size=1000)
cov_matrix_1000 = np.cov(simulations_1000.T)

np.linalg.eigvals(cov_matrix_1000)

In [ ]:
# 6.1 Brownian Motion Simulation: Iterated Scheme

def iterated_scheme():
    n = 1000
    timepoints = np.linspace(0, 1, n + 1)
    time = timepoints[1] - timepoints[0]
    increments = np.random.normal(0, np.sqrt(time), n)

    W = np.cumsum(increments)
    W = np.concatenate(([0], W))
    plt.plot(timepoints, W)
    plt.title('Iterated Scheme')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.show()

iterated_scheme()

In [ ]:
# 6.2 Cholesky Decomposition

def cholesky():
    n = 100
    timepoints = np.linspace(0.0000000001, 1, n)
    time = timepoints[1] - timepoints[0]

    covariance = np.minimum.outer(timepoints, timepoints)

    decomp = np.linalg.cholesky(covariance)

    # Generate a sample path of Brownian motion
    increments = np.random.normal(0, np.sqrt(time), n)
    value = np.dot(decomp, increments)
    value = np.cumsum(value)

    plt.plot(timepoints, value)
    plt.title('Cholesky Decomposition')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.show()

cholesky()

In [ ]:
# 6.3 (a)

def mother(t):
    if 0 <= t < 0.5:
        return 1
    if 0.5 < t <= 1:
        return -1 
    return 0

def daughter(n, t):
    j = math.floor(math.log(n, 2))
    k = n - 2**j
    new_t = (2**j)*t - k
    x = 2**(j/2)
    return (x * mother(new_t))

def wavelets(n):
    x = 1000
    time = np.linspace (0, 1, x)
    daughters = [daughter(n, time[t]) for t in range(x)]
    plt.plot(time, daughters)
    plt.title('Wavelets')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.show()

samples = [1, 5, 10, 15, 20]
for sample in samples:
    wavelets(sample)


In [ ]:
# 6.3 (b)

def mother(t):
    if 0 <= t < 0.5:
        return 2*t
    if 0.5 < t <= 1:
        return 2*(1 -t)
    return 0

def daughter(n, t):
    j = math.floor(math.log(n, 2))
    k = n - 2**j
    new_t = (2**j)*t - k
    x = 2**(j/2)
    return (x * mother(new_t))

def wavelets(n):
    x = 1000
    time = np.linspace (0, 1, x)
    daughters = [daughter(n, time[t]) for t in range(x)]
    plt.plot(time, daughters)
    plt.title('Wavelets')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.show()

for x in range(1, 33):
    wavelets(x)
